# Smoother

In [4]:
import os
import numpy as np
from scipy.ndimage import gaussian_filter1d

# Path to the uploaded zip file
output_folder_path =r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham/new_sham_zscored'

# Function to smooth trials
def smooth_trials(data_per_animal, trial_types, sigma=20):
    smoothed_trials = {}
    for animal, trials in data_per_animal.items():
        animal_trials = {}
        for trial in trial_types:
            if trial in trials:
                # Apply Gaussian smoothing to each trial
                smoothed_data = gaussian_filter1d(trials[trial], sigma=sigma)
                animal_trials[trial] = smoothed_data
        smoothed_trials[animal] = animal_trials
    return smoothed_trials

# Process each npy file in all subfolders
for root, dirs, files in os.walk(extract_folder_path):
    for filename in files:
        if filename.endswith('.npy'):
            # Load the data
            file_path = os.path.join(root, filename)
            data = np.load(file_path, allow_pickle=True).item()
            
            # Get all trial types from the data
            trial_types = list(data.keys())
            
            # Create a dictionary with the original data structure
            input_data = {filename: data}
            
            # Smooth the trials
            smoothed_data = smooth_trials(input_data, trial_types)
            
            # Extract the smoothed data (removing the filename key)
            smoothed_dict = smoothed_data[filename]
            
            # Save the smoothed data
            output_filename = f'smoothed_{filename}'
            output_path = os.path.join(root, output_filename)
            np.save(output_path, smoothed_dict)

print("Smoothing completed. Files saved with 'smoothed_' prefix.")

Smoothing completed. Files saved with 'smoothed_' prefix.


# Trimmer

In [2]:
import os
import numpy as np

# Define the base directory path
base_dir = r"C:\files\data\sensory_stim\trim"

# Define trimming parameters
sample_rate = 130  # Hz
trim_start = 5 * sample_rate  # Samples to trim from the start
trim_end = 10 * sample_rate  # Samples to trim from the end

# Walk through all directories and files in the base directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".npy") and "_trimmed" not in file:
            # Construct full file path
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            
            try:
                # Load the data with allow_pickle=True
                data = np.load(file_path, allow_pickle=True).item()
                
                # Trim each trace array in the dictionary
                for key in data.keys():
                    array = data[key]
                    # Check if the array has enough samples
                    if len(array) < trim_start + trim_end:
                        print(f"Warning: {file_path} - Array for key '{key}' is too short to trim.")
                        continue  # Skip trimming this array
                    trimmed_array = array[trim_start:-trim_end]
                    data[key] = trimmed_array
                
                # Create new file path with _trimmed suffix
                file_name, ext = os.path.splitext(file)
                new_file_name = f"{file_name}_trimmed{ext}"
                new_file_path = os.path.join(root, new_file_name)
                
                # Save the trimmed data
                np.save(new_file_path, data)
                print(f"Trimmed data saved to: {new_file_path}")
            
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

Processing file: C:\files\data\sensory_stim\trim\sham\1489_sham.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham\1489_sham_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham\1490_sham.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham\1490_sham_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham\1491_sham.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham\1491_sham_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham\1492_sham.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham\1492_sham_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham_ctrl\1489_ctrl.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham_ctrl\1489_ctrl_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham_ctrl\1490_ctrl.npy
Trimmed data saved to: C:\files\data\sensory_stim\trim\sham_ctrl\1490_ctrl_trimmed.npy
Processing file: C:\files\data\sensory_stim\trim\sham_ctrl\1491_ctrl.npy
Trimmed

# Z-scorer

In [1]:
import os
import numpy as np

# Path to the uploaded zip file
# extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham/'
# output_folder_path = r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham_zscored/'

# r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\SNI/new_SNI/'
# r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham/new_sham/'

extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\npy_files/'
output_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\npy_files\new_batch_zscored/'

# Function to z-score the data based on the baseline (first 5 seconds)
def z_score_trace(trace, baseline_start, baseline_end):
    baseline = trace[baseline_start:baseline_end]
    baseline_mean = np.mean(baseline)
    baseline_std = np.std(baseline)
    return (trace - baseline_mean) / baseline_std

# Function to z-score trials
def z_score_trials(data_per_animal, trial_types, baseline_duration=5, sampling_rate=100):
    z_scored_trials = {}
    baseline_end = int(baseline_duration * sampling_rate)
    
    for animal, trials in data_per_animal.items():
        animal_trials = {}
        for trial in trial_types:
            if trial in trials:
                # Apply z-scoring to each trial
                z_scored_data = z_score_trace(trials[trial], 0, baseline_end)
                animal_trials[trial] = z_scored_data
        z_scored_trials[animal] = animal_trials
    return z_scored_trials

# Process each npy file in all subfolders
for root, dirs, files in os.walk(extract_folder_path):
    for filename in files:
        if filename.endswith('.npy'):
            # Load the data
            file_path = os.path.join(root, filename)
            data = np.load(file_path, allow_pickle=True).item()
            
            # Get all trial types from the data
            trial_types = list(data.keys())
            
            # Create a dictionary with the original data structure
            input_data = {filename: data}
            
            # Z-score the trials
            z_scored_data = z_score_trials(input_data, trial_types)
            
            # Extract the z-scored data (removing the filename key)
            z_scored_dict = z_scored_data[filename]
            
            # Create the corresponding output directory structure
            relative_path = os.path.relpath(root, extract_folder_path)
            output_dir = os.path.join(output_folder_path, relative_path)
            os.makedirs(output_dir, exist_ok=True)
            
            # Save the z-scored data
            output_filename = f'z_scored_{filename}'
            output_path = os.path.join(output_dir, output_filename)
            np.save(output_path, z_scored_dict)

print("Z-scoring completed. Files saved with 'z_scored_' prefix in the corresponding subfolders under 'grabda_zscored'.")

Z-scoring completed. Files saved with 'z_scored_' prefix in the corresponding subfolders under 'grabda_zscored'.
